Import Numpy and Sep

In [ ]:
import numpy as np
import sep

Additional setup for reading the test image and displaying plots

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits

rcParams['figure.figsize'] = [10.,8.]

Read image into standard 2-d numpy array

In [ ]:
data = "../Astro 119 Final/image.fits"
hdu_list = fits.open(data)
data = hdu_list[0].data

Show the image

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

Save the figure as a png

In [ ]:
plt.imsave("data.png", data)

Measure a spatially varying background on the image

In [ ]:
bkg = sep.Background(data)

Get a "global" mean and noise of the image background:

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

Evaluate background as 2-d array, same size as original image

In [ ]:
bkg_image = bkg.back()

Show the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

Save the figure as a png

In [ ]:
plt.imsave("background.png", bkg_image)

Evaluate the background noise as 2-d array, same size as original image

In [ ]:
bkg_rms = bkg.rms()

Show the background noise

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

Save the figure as a png

In [ ]:
plt.imsave("noise.png", bkg_rms)

Subtract the background

In [ ]:
data_sub = data - bkg

Set detection threshold to be a constant value of 1.5rms

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

How many objects were detected

In [ ]:
len(objects)

Plot background-subtracted image and plot an ellipse for each object

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), 
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

Save the image as a png

In [ ]:
plt.imsave("objects.png", data_sub)

Available fields

In [ ]:
objects.dtype.names

Perform circular aperture photometry with a 3 pixel radius at the locations of the objects

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

Show the first 10 object results:

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))